**Właściciel tego notatnika: Tadejro**

Link do oryginalnego notatnika: https://colab.research.google.com/drive/18ykX7r5lFYSlHEAjiNfbfWyIex7WRYpk?usp=sharing

Link do źródła w repozytorium: https://github.com/8tm/mekatron-files/blob/master/colab_notepads/tacotron2/tacotron2_training.ipynb

## Zanim zaczniesz to [kliknij tu i przeczytaj wszystko](https://colab.research.google.com/drive/1wcAU5nNtFXas6euBA9Hf-jCtygZDWET9)

In [ ]:
#@title # Informacje o karcie graficznej i wersję pythona
_, card = !nvidia-smi --query-gpu=gpu_name --format='csv'
print(card)
!python -V


In [ ]:
#@title #Instalacja paczek
%%sh
echo "Packages for Python 3.10.12"
pip install -q \
    gdown==4.7.3 \
    unidecode==1.3.6 \
    tensorboardX==2.6.1 \
    pyunpack==0.3 \
    patool==1.12 \
    pynvml==11.5.0 \
    librosa==0.10.0-2


In [ ]:
#@title Łączenie się z dyskiem nowym sposobem (możliwe tylko na swoje konto Google)
from google.colab import drive
import os
drive.mount('drive', force_remount=True)

In [ ]:
#@markdown #Wstępna inicjalizacja
#@markdown **voice_name** - Nazwa folderu w folderze **mekatron** na dysku, gdzie są potrzebne dane + nazwa wyjściowego modelu mekatrona

voice_name = "Carrie" #@param{type:"string"}
main_folder = '/content/drive/MyDrive/mekatron'

if not os.path.exists(f'{main_folder}/{voice_name}'):
    raise NotADirectoryError('Nie ma takiego folderu. Utwórz go i przenieś co potrzebne.')
#@markdown **train_list_name** - Nazwa listy treningowych plików dźwiękowych
train_list_name = "list_train.txt" #@param{type:'string'}
#@markdown **validation_list_name** - Nazwa listy walidacyjnych plików dźwiękowych

validation_list_name = "list_val.txt" #@param{type:'string'}
#@markdown **archive_name** - Nazwa archiwum z plikami dźwiękowymi.
archive_name = "wavs.7z" #@param{type:'string'}


names_list = [voice_name, validation_list_name, train_list_name, archive_name]

for i in names_list:
    if i == "":
        raise ValueError("Wszystkie ciągi znaków muszą być wypełnione")
    if not os.path.exists(f"{main_folder}/{voice_name}/{i}") and i != voice_name:
        raise FileNotFoundError(f"{main_folder}/{voice_name}/{i}: Nie znaleziono podanego pliku.")
print(f"Ścieżka do folderu głównego: {main_folder}/{voice_name}")
print(f"Ścieżka do pliku z listą plików walidacyjnych: {main_folder}/{voice_name}/{validation_list_name}")
print(f"Ścieżka do pliku z listą plików treningowych: {main_folder}/{voice_name}/{train_list_name}")
print(f"Ścieżka do archiwum z plikami dźwiękowymi: {main_folder}/{voice_name}/{archive_name}")
print(f"Ścieżka do pliku wyjściowego: {main_folder}/{voice_name}/{voice_name}.tt2")

In [ ]:
#@title # Import bibliotek, załadowanie własnych funkcji

import argparse
import glob
import locale
import math
import os
import random
import sys
import time
import warnings
from distutils.dir_util import copy_tree
from math import e

# from apex import amp  # Naprawić - dodać instalację amp'a z repo
import gdown
import matplotlib
import matplotlib.pylab as plt
import numpy as np
import torch
import torch.distributed as dist
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler
from pynvml import *
from pyunpack import Archive
from numpy import finfo
from tqdm.notebook import tqdm


def git_clone(url):
    project_name = os.path.splitext(os.path.basename(url))[0]
    if not os.path.exists(project_name):
        !git clone -q --recursive {url}
    return project_name


project_name = git_clone('https://github.com/8tm/mekatron2.git')
os.chdir(f'/content/{project_name}')
!mkdir "filelists"
!cp {main_folder}/{voice_name}/{train_list_name} 'filelists'
!cp {main_folder}/{voice_name}/{validation_list_name} 'filelists'
!git submodule init
!git submodule update

sys.path.append(os.path.join(project_name, 'waveglow/'))
sys.path.append(project_name)


from distributed import apply_gradient_allreduce
import layers
from model import Tacotron2
from data_utils import TextMelLoader, TextMelCollate
from loss_function import Tacotron2Loss
from logger import Tacotron2Logger
from hparams import create_hparams
from utils import load_wav_to_torch, load_filepaths_and_text
from text import text_to_sequence


def getpreferredencoding(do_setlocale = True):
    return "UTF-8"


def download_from_google_drive(file_id, file_name, quiet=False, fuzzy=True):
    gdown.download(
        f'https://drive.google.com/uc?id={file_id.strip()}',
        file_name,
        quiet=quiet,
        fuzzy=fuzzy,
    )


def create_mels():
    print("Generating Mels")
    stft = layers.TacotronSTFT(
                hparams.filter_length, hparams.hop_length, hparams.win_length,
                hparams.n_mel_channels, hparams.sampling_rate, hparams.mel_fmin,
                hparams.mel_fmax)
    def save_mel(filename):
        audio, sampling_rate = load_wav_to_torch(filename)
        if sampling_rate != stft.sampling_rate:
            return
            #raise ValueError("{} {} SR doesn't match target {} SR".format(filename,
            #    sampling_rate, stft.sampling_rate))
        audio_norm = audio / hparams.max_wav_value
        audio_norm = audio_norm.unsqueeze(0)
        audio_norm = torch.autograd.Variable(audio_norm, requires_grad=False)
        melspec = stft.mel_spectrogram(audio_norm)
        melspec = torch.squeeze(melspec, 0).cpu().numpy()
        
        filename = filename.rsplit('.', 1)[0]
        np.save(filename, melspec)

    files = glob.glob('wavs/*.*')
    for i in tqdm(files):
        filename, extension = os.path.splitext(i)
        if extension.lower() == '.wav':
            save_mel(i)


def reduce_tensor(tensor, n_gpus):
    rt = tensor.clone()
    dist.all_reduce(rt, op=dist.reduce_op.SUM)
    rt /= n_gpus
    return rt


def init_distributed(hparams, n_gpus, rank, group_name):
    assert torch.cuda.is_available(), "Distributed mode requires CUDA."
    print("Initializing Distributed")

    torch.cuda.set_device(rank % torch.cuda.device_count())

    dist.init_process_group(
        backend=hparams.dist_backend, init_method=hparams.dist_url,
        world_size=n_gpus, rank=rank, group_name=group_name)

    print("Done initializing distributed")


def prepare_dataloaders(hparams):
    # Get data, data loaders and collate function ready
    trainset = TextMelLoader(hparams.training_files, hparams)
    valset = TextMelLoader(hparams.validation_files, hparams)
    collate_fn = TextMelCollate(hparams.n_frames_per_step)

    if hparams.distributed_run:
        train_sampler = DistributedSampler(trainset)
        shuffle = False
    else:
        train_sampler = None
        shuffle = True

    train_loader = DataLoader(trainset, num_workers=1, shuffle=shuffle,
                              sampler=train_sampler,
                              batch_size=hparams.batch_size, pin_memory=False,
                              drop_last=True, collate_fn=collate_fn)
    return train_loader, valset, collate_fn


def prepare_directories_and_logger(output_directory, log_directory, rank):
    if rank == 0:
        if not os.path.isdir(output_directory):
            os.makedirs(output_directory)
            os.chmod(output_directory, 0o775)
        logger = Tacotron2Logger(os.path.join(output_directory, log_directory))
    else:
        logger = None
    return logger


def load_model(hparams):
    model = Tacotron2(hparams).cuda()
    if hparams.fp16_run:
        model.decoder.attention_layer.score_mask_value = finfo('float16').min

    if hparams.distributed_run:
        model = apply_gradient_allreduce(model)

    return model


def warm_start_model(checkpoint_path, model, ignore_layers):
    assert os.path.isfile(checkpoint_path)
    print("Warm starting model from checkpoint '{}'".format(checkpoint_path))
    checkpoint_dict = torch.load(checkpoint_path, map_location='cpu')
    model_dict = checkpoint_dict['state_dict']
    if len(ignore_layers) > 0:
        model_dict = {k: v for k, v in model_dict.items()
                      if k not in ignore_layers}
        dummy_dict = model.state_dict()
        dummy_dict.update(model_dict)
        model_dict = dummy_dict
    model.load_state_dict(model_dict)
    return model


def load_checkpoint(checkpoint_path, model, optimizer):
    assert os.path.isfile(checkpoint_path)
    print("Loading checkpoint '{}'".format(checkpoint_path))
    checkpoint_dict = torch.load(checkpoint_path, map_location='cpu')
    model.load_state_dict(checkpoint_dict['state_dict'])
    optimizer.load_state_dict(checkpoint_dict['optimizer'])
    learning_rate = checkpoint_dict['learning_rate']
    iteration = checkpoint_dict['iteration']
    print("Loaded checkpoint '{}' from iteration {}" .format(
        checkpoint_path, iteration))
    return model, optimizer, learning_rate, iteration


def save_checkpoint(model, optimizer, learning_rate, iteration, filepath):
    print("Saving model and optimizer state at iteration {} to {}".format(iteration, filepath))
    try:
        torch.save({'iteration': iteration,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'learning_rate': learning_rate}, filepath)
    except KeyboardInterrupt:
        print("interrupt received while saving, waiting for save to complete.")
        torch.save({'iteration': iteration,'state_dict': model.state_dict(),'optimizer': optimizer.state_dict(),'learning_rate': learning_rate}, filepath)
    print("Model Saved")


def plot_alignment(alignment, info=None):
    %matplotlib inline
    fig, ax = plt.subplots(figsize=(int(alignment_graph_width/100), int(alignment_graph_height/100)))
    im = ax.imshow(alignment, cmap='inferno', aspect='auto', origin='lower',
                   interpolation='none')
    ax.autoscale(enable=True, axis="y", tight=True)
    fig.colorbar(im, ax=ax)
    xlabel = 'Decoder timestep'
    if info is not None:
        xlabel += '\n\n' + info
    plt.xlabel(xlabel)
    plt.ylabel('Encoder timestep')
    plt.tight_layout()
    fig.canvas.draw()
    plt.show()


def validate(model, criterion, valset, iteration, batch_size, n_gpus,
             collate_fn, logger, distributed_run, rank, epoch, start_eposh, learning_rate):
    """Handles all the validation scoring and printing"""
    model.eval()
    with torch.no_grad():
        val_sampler = DistributedSampler(valset) if distributed_run else None
        val_loader = DataLoader(valset, sampler=val_sampler, num_workers=1,
                                shuffle=False, batch_size=batch_size,
                                pin_memory=False, collate_fn=collate_fn)

        val_loss = 0.0
        for i, batch in enumerate(val_loader):
            x, y = model.parse_batch(batch)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            if distributed_run:
                reduced_val_loss = reduce_tensor(loss.data, n_gpus).item()
            else:
                reduced_val_loss = loss.item()
            val_loss += reduced_val_loss
        val_loss = val_loss / (i + 1)

    model.train()
    if rank == 0:
        print("Epoch: {} Validation loss {}: {:9f}  Time: {:.1f}m LR: {:.6f}".format(epoch, iteration, val_loss,(time.perf_counter()-start_eposh)/60, learning_rate))
        logger.log_validation(val_loss, model, y, y_pred, iteration)
        if hparams.show_alignments:
            %matplotlib inline
            _, mel_outputs, gate_outputs, alignments = y_pred
            idx = random.randint(0, alignments.size(0) - 1)
            plot_alignment(alignments[idx].data.cpu().numpy().T)


def train(output_directory, log_directory, checkpoint_path, warm_start, n_gpus,
          rank, group_name, hparams, log_directory2):
    """Training and validation logging results to tensorboard and stdout

    Params
    ------
    output_directory (string): directory to save checkpoints
    log_directory (string) directory to save tensorboard logs
    checkpoint_path(string): checkpoint path
    n_gpus (int): number of gpus
    rank (int): rank of current gpu
    hparams (object): comma separated list of "name=value" pairs.
    """
    if hparams.distributed_run:
        init_distributed(hparams, n_gpus, rank, group_name)

    torch.manual_seed(hparams.seed)
    torch.cuda.manual_seed(hparams.seed)

    model = load_model(hparams)
    learning_rate = hparams.learning_rate
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
                                 weight_decay=hparams.weight_decay)

    if hparams.fp16_run:
        model, optimizer = amp.initialize(
            model, optimizer, opt_level='O2')

    if hparams.distributed_run:
        model = apply_gradient_allreduce(model)

    criterion = Tacotron2Loss()

    logger = prepare_directories_and_logger(output_directory, log_directory, rank)

    train_loader, valset, collate_fn = prepare_dataloaders(hparams)

    # Load checkpoint if one exists
    iteration = 0
    epoch_offset = 0
    #@markdown **epochs_to_save** określa, co ile epok będzie zapisywany postęp treningu.
    epochs_to_save =  150 #@param{type:'number'}
    if checkpoint_path is not None and os.path.isfile(checkpoint_path):
        if warm_start:
            model = warm_start_model(checkpoint_path, model, hparams.ignore_layers)
        else:
            model, optimizer, _learning_rate, iteration = load_checkpoint(
                checkpoint_path, model, optimizer)
            if hparams.use_saved_learning_rate:
                learning_rate = _learning_rate
            iteration += 1  # next iteration is iteration + 1
            epoch_offset = max(0, int(iteration / len(train_loader)))
    else:
      os.path.isfile("pretrained_model")
      #@markdown Tutaj podajesz ID  modelu od którego zaczynasz trening
      pretrained_model_id = "1sv0oAfCIpXdI-kUB_1qDrPZlJ5KD65Pt" #@param{type:'string'}
      download_from_google_drive(pretrained_model_id ,"pretrained_model")
      model = warm_start_model("pretrained_model", model, hparams.ignore_layers)
      # download LJSpeech pretrained model if no checkpoint already exists

    start_eposh = time.perf_counter()
    learning_rate = 0.0
    model.train()
    is_overflow = False
    #@markdown **Tryb uproszczony** - Zmniejsza ilość informacji na wyjściu do koniecznego minimum
    simplified_view = True #@param{type:'boolean'}
    # ================ MAIN TRAINNIG LOOP! ===================
    if not simplified_view:
        for epoch in tqdm(range(epoch_offset, hparams.epochs)):
            print("\nStarting Epoch: {} Iteration: {}".format(epoch, iteration))
            start_eposh = time.perf_counter() # eposh is russian, not a typo
            for i, batch in tqdm(enumerate(train_loader), total=len(train_loader)):
                start = time.perf_counter()
                if iteration < hparams.decay_start: learning_rate = hparams.A_
                else: iteration_adjusted = iteration - hparams.decay_start; learning_rate = (hparams.A_*(e**(-iteration_adjusted/hparams.B_))) + hparams.C_
                learning_rate = max(hparams.min_learning_rate, learning_rate) # output the largest number
                for param_group in optimizer.param_groups:
                    param_group['lr'] = learning_rate

                model.zero_grad()
                x, y = model.parse_batch(batch)
                y_pred = model(x)

                loss = criterion(y_pred, y)
                if hparams.distributed_run:
                    reduced_loss = reduce_tensor(loss.data, n_gpus).item()
                else:
                    reduced_loss = loss.item()

                if hparams.fp16_run:
                    with amp.scale_loss(loss, optimizer) as scaled_loss:
                        scaled_loss.backward()
                else:
                    loss.backward()

                if hparams.fp16_run:
                    grad_norm = torch.nn.utils.clip_grad_norm_(
                        amp.master_params(optimizer), hparams.grad_clip_thresh)
                    is_overflow = math.isnan(grad_norm)
                else:
                    grad_norm = torch.nn.utils.clip_grad_norm_(
                        model.parameters(), hparams.grad_clip_thresh)

                optimizer.step()

                if not is_overflow and rank == 0:
                    duration = time.perf_counter() - start
                    logger.log_training(
                        reduced_loss, grad_norm, learning_rate, duration, iteration)
                    #print("Batch {} loss {:.6f} Grad Norm {:.6f} Time {:.6f}".format(iteration, reduced_loss, grad_norm, duration), end='\r', flush=True)

                iteration += 1
            validate(model, criterion, valset, iteration,
                    hparams.batch_size, n_gpus, collate_fn, logger,
                    hparams.distributed_run, rank, epoch, start_eposh, learning_rate)
            if epoch % epochs_to_save == 0:
                save_checkpoint(model, optimizer, learning_rate, iteration, checkpoint_path)
            if log_directory2 != None:
                copy_tree(log_directory, log_directory2)
    else:
        for epoch in range(epoch_offset, hparams.epochs):
            print("\nStarting Epoch: {} Iteration: {}".format(epoch, iteration))
            for i, batch in enumerate(train_loader):
                start = time.perf_counter()
                if iteration < hparams.decay_start: learning_rate = hparams.A_
                else: iteration_adjusted = iteration - hparams.decay_start; learning_rate = (hparams.A_*(e**(-iteration_adjusted/hparams.B_))) + hparams.C_
                learning_rate = max(hparams.min_learning_rate, learning_rate) # output the largest number
                for param_group in optimizer.param_groups:
                    param_group['lr'] = learning_rate

                model.zero_grad()
                x, y = model.parse_batch(batch)
                y_pred = model(x)

                loss = criterion(y_pred, y)
                if hparams.distributed_run:
                    reduced_loss = reduce_tensor(loss.data, n_gpus).item()
                else:
                    reduced_loss = loss.item()
                if hparams.fp16_run:
                    with amp.scale_loss(loss, optimizer) as scaled_loss:
                        scaled_loss.backward()
                else:
                    loss.backward()

                if hparams.fp16_run:
                    grad_norm = torch.nn.utils.clip_grad_norm_(
                        amp.master_params(optimizer), hparams.grad_clip_thresh)
                    is_overflow = math.isnan(grad_norm)
                else:
                    grad_norm = torch.nn.utils.clip_grad_norm_(
                        model.parameters(), hparams.grad_clip_thresh)

                optimizer.step()

                if not is_overflow and rank == 0:
                    duration = time.perf_counter() - start
                    logger.log_training(
                        reduced_loss, grad_norm, learning_rate, duration, iteration)
                    #print("Batch {} loss {:.6f} Grad Norm {:.6f} Time {:.6f}".format(iteration, reduced_loss, grad_norm, duration), end='\r', flush=True)

                iteration += 1
            validate(model, criterion, valset, iteration,
                    hparams.batch_size, n_gpus, collate_fn, logger,
                    hparams.distributed_run, rank, epoch, start_eposh, learning_rate)
            if epoch % epochs_to_save == 0:
                save_checkpoint(model, optimizer, learning_rate, iteration, checkpoint_path)
            if log_directory2 != None:
                copy_tree(log_directory, log_directory2)

def check_dataset(hparams):
    def check_arr(filelist_arr):
        for i, file in enumerate(filelist_arr):
            if len(file) > 2:
                print("|".join(file), "\nhas multiple '|', this may not be an error.")
            if hparams.load_mel_from_disk and '.wav' in file[0]:
                print("[WARNING]", file[0], " in filelist while expecting .npy .")
            else:
                if not hparams.load_mel_from_disk and '.npy' in file[0]:
                    print("[WARNING]", file[0], " in filelist while expecting .wav .")
            if (not os.path.exists(file[0])):
                print("|".join(file), "\n[WARNING] does not exist.")
            if len(file[1]) < 3:
                print("|".join(file), "\n[info] has no/very little text.")
            if not ((file[1].strip())[-1] in r"!?,.;:"):
                print("|".join(file), "\n[info] has no ending punctuation.")
            mel_length = 1
            if hparams.load_mel_from_disk and '.npy' in file[0]:
                melspec = torch.from_numpy(np.load(file[0], allow_pickle=True))
                mel_length = melspec.shape[1]
            if mel_length == 0:
                print("|".join(file), "\n[WARNING] has 0 duration.")
    print("Checking Training Files")
    audiopaths_and_text = load_filepaths_and_text(hparams.training_files) # get split lines from training_files text file.
    check_arr(audiopaths_and_text)
    print("Checking Validation Files")
    audiopaths_and_text = load_filepaths_and_text(hparams.validation_files) # get split lines from validation_files text file.
    check_arr(audiopaths_and_text)
    print("Finished Checking")


locale.getpreferredencoding = getpreferredencoding



In [ ]:
#@title # Tworzenie folderu wavs
data_path = '/content/mekatron2/wavs'
!mkdir {data_path}

In [ ]:
#@title # Rozpakowanie plików
archive_path = f'/content/drive/MyDrive/mekatron/{voice_name}/{archive_name}'
Archive(archive_path).extractall('/content/mekatron2/wavs')

In [ ]:
#@title # Ścieżka do modelu #
%matplotlib inline


if os.getcwd() != '/content/mekatron2':
    os.chdir('/content/mekatron2')


warnings.filterwarnings("ignore", message=r".*The given NumPy array is not writeable.*", category=UserWarning)


# ---- Replace .wav with .npy in filelists ----
!sed -i -- 's,.wav|,.npy|,Ig' filelists/*.txt
# ---- Replace .wav with .npy in filelists ----

n_gpus=1
rank=0
group_name=None

# ---- DEFAULT PARAMETERS DEFINED HERE ----
hparams = create_hparams()
model_filename = 'current_model'
hparams.training_files = "filelists/clipper_train_filelist.txt"
hparams.validation_files = "filelists/clipper_val_filelist.txt"
#hparams.use_mmi=True,          # not used in this notebook
#hparams.use_gaf=True,          # not used in this notebook
#hparams.max_gaf=0.5,           # not used in this notebook
#hparams.drop_frame_rate = 0.2  # not used in this notebook
hparams.p_attention_dropout=0.1
hparams.p_decoder_dropout=0.1
hparams.decay_start = 15000
hparams.A_ = 5e-4
hparams.B_ = 8000
hparams.C_ = 0
hparams.min_learning_rate = 1e-5
generate_mels = True
hparams.show_alignments = True
alignment_graph_height = 600
alignment_graph_width = 1000
hparams.batch_size = 32
hparams.load_mel_from_disk = True
hparams.ignore_layers = []
hparams.epochs = 10000
torch.backends.cudnn.enabled = hparams.cudnn_enabled
torch.backends.cudnn.benchmark = hparams.cudnn_benchmark
output_directory = f'/content/drive/MyDrive/mekatron/{voice_name}/{voice_name}'  # Location to save Checkpoints
log_directory = '/content/tacotron2/logs' # Location to save Log files locally
log_directory2 = '/content/drive/My Drive/colab/logs/Astrid' # Location to copy log files (done at the end of each epoch to cut down on I/O)
checkpoint_path = output_directory+(r'/')+model_filename
#@markdown ---

In [ ]:
#@title # Warm Start #
#@markdown Jeżeli zaznaczone, to trening zacznie się od nowa na bazie pretrenowanego modelu.

#@markdown **Zaznaczyć**, jeżeli trening jest zaczynany

#@markdown **Odznaczyć, jeżeli trening jest kontynuowany.**
warm_start=True #@param{type:"boolean"}
#@markdown ---

In [ ]:
#@title # Konfiguracja list
hparams.training_files = f"/content/mekatron2/filelists/list_train.txt"
hparams.validation_files = f"/content/mekatron2/filelists/list_val.txt"

In [ ]:
#@title # Prosta konfiguracja treningu #
# hparams to Tune

# These two are the most important
#@markdown **batch_size** odpowiada za to, ile weźmie algorytm próbek naraz, co wpływa na szybkość treningu.

#@markdown Jeżeli będzie zbyt duży, to skrypt się wysypie z powodu braku pamięci i trzeba będzie kliknąć Runtime -> Restart Runtime oraz zmniejszyć ten parametr.

#@markdown Wyznaczany eksperymentalnie. K80 ma mniej pamięci niż T4, więc mniejszy batch_size trzeba ustawić.

#@markdown Jeżeli batch_size przekracza ilość wpisów listy treningowej, to batch_size będzie równy ilości wpisów do zbioru treningowego
hparams.batch_size =  6#@param{type:'number', min:0}
max_length = len(list(iter(open("./filelists/list_train.txt").readlines())))

if  hparams.batch_size > max_length:
    hparams.batch_size = max_length
#@markdown **epochs** oznacza, ile razy AI będzie zaglądać do plików zanim automatycznie skończy trening.


#@markdown Najlepiej na bieżąco monitorować w syntezatorze rezultaty i robić kopie zapasowe tych, które brzmią sensownie jak na długość datasetu.
hparams.epochs =  2000#@param{type:'number', min:0, max:99999}

#The rest aren't that important
hparams.p_attention_dropout=0.1
hparams.p_decoder_dropout=0.1
hparams.decay_start = 15000         # wait till decay_start to start decaying learning rate
hparams.A_ = 5e-4                   # Start/Max Learning Rate 5e-4 base
hparams.B_ = 8000                   # Decay Rate
hparams.C_ = 0                      # Shift learning rate equation by this value
hparams.min_learning_rate = 1e-5    # Min Learning Rate
generate_mels = True # Don't change
hparams.show_alignments = True
alignment_graph_height = 600
alignment_graph_width = 1000
hparams.load_mel_from_disk = True
hparams.ignore_layers = [] # Layers to reset (None by default, other than foreign languages this param can be ignored)

torch.backends.cudnn.enabled = hparams.cudnn_enabled
torch.backends.cudnn.benchmark = hparams.cudnn_benchmark
output_directory = '/content/drive/My Drive/colab/outdir/Delvin' # Location to save Checkpoints
log_directory = '/content/tacotron2/logs' # Location to save Log files locally
log_directory2 = '/content/drive/My Drive/colab/logs/Delvin' # Location to copy log files (done at the end of each epoch to cut down on I/O)
#@markdown ---

In [ ]:
#@title # Ustawienie ściezki
checkpoint_path = f'/content/drive/MyDrive/mekatron/{voice_name}/{voice_name}.tt2'


In [ ]:
#@title # Generowanie Meli
if generate_mels:
    create_mels()


In [ ]:
#@title # Weryfikacja datasetu
check_dataset(hparams)

In [ ]:
#@title # Trening
print('FP16 Run:', hparams.fp16_run)
print('Dynamic Loss Scaling:', hparams.dynamic_loss_scaling)
print('Distributed Run:', hparams.distributed_run)
print('cuDNN Enabled:', hparams.cudnn_enabled)
print('cuDNN Benchmark:', hparams.cudnn_benchmark)

import torch, gc

gc.collect()
torch.cuda.empty_cache()

train(output_directory, log_directory, checkpoint_path, warm_start, n_gpus, rank, group_name, hparams, log_directory2)

In [ ]:
#@title # Podsumowanie

nvmlInit()
h = nvmlDeviceGetHandleByIndex(0)
info = nvmlDeviceGetMemoryInfo(h)
print(f'total    : {info.total}')
print(f'free     : {info.free}')
print(f'used     : {info.used}')
print(f'used per batch_size: {info.used / hparams.batch_size / 1024**3}')
